In [1]:
import pandas as pd
import numpy as np
import sys
import os

In [2]:
path = os.path.dirname(os.path.abspath('__file__'))+'\\Prepared_tables\\'
df = pd.read_csv(path + 'Prep_Protein Groups_cut05_notimuted.csv', low_memory=False)

In [3]:
def get_mito_annotation(df):    
    mitochondrial = []
    for i in range(len(df)):
        if 'Verified mitochondrial' in df['IMPI_new'][i] or str(df['SubMitoLocalization'][i])!='nan':
            mitochondrial.append('Mitochondrial')
        elif 'Ancillary' in df['IMPI_new'][i] or 'Associated' in df['IMPI_new'][i]:
            mitochondrial.append('Associated')
        else:
            mitochondrial.append('Not Mitochondrial')
            
    return mitochondrial


In [6]:
df['StringID'] = [list(str(df['StringID'][i]).split(';')) for i in range(len(df))]
df['IMPI_new'] = [list(str(df['IMPI_new'][i]).split(';')) for i in range(len(df))]
df['Majority protein IDs'] = [list(str(df['Majority protein IDs'][i]).split(';')) for i in range(len(df))]
df['ESNG'] = [list(str(df['ESNG'][i]).split(';')) for i in range(len(df))]
df['Entrez_GeneID'] = [list(str(df['Entrez_GeneID'][i]).split(';')) for i in range(len(df))]


df['Cyto_ID'] =  [[df['StringID'][x][i] for i in range(len(df['StringID'][x])) if df['StringID'][x][i] !=''][0] if list(set(df['StringID'][x])) != [''] else 'nan' for x in range(len(df))]
df['Cyto_IMPI'] = [[df['IMPI_new'][x][i] for i in range(len(df['StringID'][x])) if df['StringID'][x][i] !=''][0] if list(set(df['StringID'][x])) != [''] else 'nan' for x in range(len(df))]
df['Cyto_ProteinID'] = [[df['Majority protein IDs'][x][i] for i in range(len(df['StringID'][x])) if df['StringID'][x][i] !=''][0] if list(set(df['StringID'][x])) != [''] else 'nan' for x in range(len(df))]
df['Cyto_ESNG'] = [[df['ESNG'][x][i] for i in range(len(df['StringID'][x])) if df['StringID'][x][i] !=''][0] if list(set(df['StringID'][x])) != [''] else 'nan' for x in range(len(df))]
df['Cyto_Entrez_GeneID'] = [[df['Entrez_GeneID'][x][i] for i in range(len(df['StringID'][x])) if df['StringID'][x][i] !=''][0] if list(set(df['StringID'][x])) != [''] else 'nan' for x in range(len(df))]

df['StringID']= [';'.join(df['StringID'][i]) for i in range(len(df))]
df['IMPI_new']= [';'.join(df['IMPI_new'][i]) for i in range(len(df))]
df['Protein IDs']= [';'.join(df['Protein IDs'][i]) for i in range(len(df))]
df['ESNG']= [';'.join(df['ESNG'][i]) for i in range(len(df))]
df['Entrez_GeneID']= [';'.join(df['Entrez_GeneID'][i]) for i in range(len(df))]


#df['Mitochondrial'] = ['Mitochondrial' if df['Cyto_IMPI'][i] == 'Known mitochondrial' or df['Cyto_IMPI'][i] == 'Predicted mitochondrial' or str(df['SubMitoLocalization'][i]) != 'nan' else 'Not Mitochondrial' for i in range(len(df)) ]
df['Mitochondrial'] = get_mito_annotation(df)
#MyList = [df['Cyto_ID'][i] for i in range(len(df)) if df['Cyto_ID'][i] != 'nan']
#MyFile=open(path+ 'Prep_ProteinGroups_allproteins_75_Cyto_input.txt','w')
#
#for element in MyList:
#     MyFile.write(element)
#     MyFile.write('\n')
#MyFile.close()

#df.to_csv((path+ 'Prep_ProteinGroups_allproteins_75.csv'), index= False)


In [7]:
df_wo_dup = df.drop_duplicates(subset=['Cyto_ID'])
df_wo_dup =df_wo_dup[df_wo_dup['Cyto_ID']!='nan'].reset_index(drop= True)

df_wo_dup['Heart_ID'] = df_wo_dup['Valid values_H_']*df_wo_dup['Values_Tissue_H_']
df_wo_dup['Heart_ID'] = df_wo_dup['Heart_ID'].astype(int)

df_wo_dup['SKM_ID'] = df_wo_dup['Valid values_SKM_']*df_wo_dup['Values_Tissue_SKM_']
df_wo_dup['SKM_ID'] = df_wo_dup['SKM_ID'].astype(int)

df_wo_dup['BAT_ID'] = df_wo_dup['Valid values_BAT_']*df_wo_dup['Values_Tissue_BAT_']
df_wo_dup['BAT_ID'] = df_wo_dup['BAT_ID'].astype(int)

df_wo_dup['Kidney_ID'] = df_wo_dup['Valid values_K_']*df_wo_dup['Values_Tissue_K_']
df_wo_dup['Kidney_ID'] = df_wo_dup['Kidney_ID'].astype(int)

df_wo_dup['Liver_ID'] = df_wo_dup['Valid values_L_']*df_wo_dup['Values_Tissue_L_']
df_wo_dup['Liver_ID'] = df_wo_dup['Liver_ID'].astype(int)

df_wo_dup['Brain_ID'] = df_wo_dup['Valid values_B_']*df_wo_dup['Values_Tissue_B_']
df_wo_dup['Brain_ID'] = df_wo_dup['Brain_ID'].astype(int)

df_wo_dup['Spleen_ID'] = df_wo_dup['Valid values_S_']*df_wo_dup['Values_Tissue_S_']
df_wo_dup['Spleen_ID'] = df_wo_dup['Spleen_ID'].astype(int)

df_wo_dup['Summed_ID'] = df_wo_dup['Heart_ID']+df_wo_dup['BAT_ID']+df_wo_dup['SKM_ID']+df_wo_dup['Spleen_ID']+df_wo_dup['Brain_ID']+df_wo_dup['Kidney_ID']+df_wo_dup['Liver_ID']
df_wo_dup['H_SKM_BAT_L_K_B_S_over5'] = (df_wo_dup['Heart_ID']>=5).astype(str)+'_'+(df_wo_dup['SKM_ID']>=5).astype(str)+'_'+(df_wo_dup['BAT_ID']>=5).astype(str)+'_'+(df_wo_dup['Liver_ID']>=5).astype(str)+'_'+(df_wo_dup['Kidney_ID']>=5).astype(str)+'_'+(df_wo_dup['Brain_ID']>=5).astype(str)+'_'+(df_wo_dup['Spleen_ID']>=5).astype(str)

In [8]:
MyList = [df_wo_dup['Cyto_ID'][i] for i in range(len(df_wo_dup)) if df_wo_dup['Cyto_ID'][i] != 'nan']
MyFile=open(path+ 'Prep_Cyto_annotated_input.txt','w')

for element in MyList:
     MyFile.write(element)
     MyFile.write('\n')
MyFile.close()

In [9]:
df_wo_dup.to_csv((path+ 'Cyto_network_annotated.csv'), index= False)